In [30]:
import pandas as pd
import json
import requests
from config import api_key

In [2]:
# Basic structure of API call:
# year.dev-category.dev-friendly-variable-name.

In [28]:
# Params
base_url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?'
state = 'school.state=FL&fields=id,school.name,school.state' 
year = 2018
cost_per = f'{year}.cost.attendance.academic_year,{year}.cost.attendance.program_year'
predom_type = 'school.degrees_awarded.predominant_recoded'

# Grab the base url
url = f'{base_url}{state},{cost_per},{predom_type}&api_key={api_key}'
response = requests.get(url).json()
response

{'metadata': {'total': 387, 'page': 0, 'per_page': 20},
 'results': [{'school.name': 'Palm Beach Academy of Health & Beauty',
   'school.state': 'FL',
   'id': 449490,
   'school.degrees_awarded.predominant_recoded': 1,
   '2018.cost.attendance.program_year': 25572,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'SABER College',
   'school.state': 'FL',
   'id': 449506,
   'school.degrees_awarded.predominant_recoded': 2,
   '2018.cost.attendance.program_year': 19233,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'Pensacola School of Massage Therapy & Health Careers',
   'school.state': 'FL',
   'id': 449533,
   'school.degrees_awarded.predominant_recoded': 1,
   '2018.cost.attendance.program_year': 21608,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'Paul Mitchell the School-Tampa',
   'school.state': 'FL',
   'id': 449959,
   'school.degrees_awarded.predominant_recoded': 1,
   '2018.cost.attendance.program_year': 21596,
   '2

#specifying areas of studies Key:
    1: Undergraduate Certificates or Diplomas
    2: Associate’s Degrees
    3: Bachelor’s Degrees
    4: Post-Baccalaureate Certificates
    5: Master’s Degrees
    6: Doctoral Degrees
    7: First Professional Degrees
    8: Graduate / Professional Certificates

In [51]:
# Create DataFrame
school_df = pd.DataFrame()

# Start of loop to append columns in DataFrame
for x in response['results']:
    df = pd.DataFrame(
            {'School Name': [x['school.name']]
            ,'School State':[x['school.state']]
            ,'Degree Type': [x['school.degrees_awarded.predominant_recoded']]
            ,'Program Cost':[x[f'{year}.cost.attendance.program_year']]
            ,'Annual Cost':[x[f'{year}.cost.attendance.academic_year']]
            }, index = [x['id']])
    school_df = school_df.append(df)

# Display data
school_df

,School Name,School State,Degree Type,Program Cost,Annual Cost
449490,Palm Beach Academy of Health & Beauty,FL,1,25572,None
449506,SABER College,FL,2,19233,None
449533,Pensacola School of Massage Therapy & Health C...,FL,1,21608,None
449959,Paul Mitchell the School-Tampa,FL,1,21596,None
40884402,Florida National University Training Center,FL,2,None,None
459310,Future-Tech Institute,FL,1,None,11429
459329,Fred K Marchman Technical College,FL,1,8557,None
481483,Boca Beauty Academy,FL,1,15907,None
482459,DeVry University-Florida,FL,3,None,31391
487393,Florida Career College-West Palm Beach,FL,1,34445,None


In [70]:
school_df['Total Cost'] = ''

for index, row in school_df.iterrows():
    if row['Annual Cost'] == None:
        row['Total Cost'] = row['Program Cost']
    # (3) = Bach Degree
    elif row['Degree Type'] == 3:
        row['Total Cost'] = row['Annual Cost']*4
    # (2) = A.A Degree
    elif row['Degree Type'] == 2:
        row['Total Cost'] = row['Annual Cost']*2
    # If not an A.A or Bach, we are assumming it is a 1 year certificate
    else:
        row['Total Cost'] = row['Annual Cost']
school_df

,School Name,School State,Degree Type,Program Cost,Annual Cost,Total Cost
449490,Palm Beach Academy of Health & Beauty,FL,1,25572,None,25572
449506,SABER College,FL,2,19233,None,19233
449533,Pensacola School of Massage Therapy & Health C...,FL,1,21608,None,21608
449959,Paul Mitchell the School-Tampa,FL,1,21596,None,21596
40884402,Florida National University Training Center,FL,2,None,None,None
459310,Future-Tech Institute,FL,1,None,11429,11429
459329,Fred K Marchman Technical College,FL,1,8557,None,8557
481483,Boca Beauty Academy,FL,1,15907,None,15907
482459,DeVry University-Florida,FL,3,None,31391,125564
487393,Florida Career College-West Palm Beach,FL,1,34445,None,34445
